In [2]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_2556/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [4]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2_sup")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [7]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_16_9_24,0.999999,0.999063,0.999999,1.000000,0.999999,6.485068e-07,0.000874,1.371015e-06,4.713938e-14,6.855074e-07,0.000045,0.000805,1.000002,0.000840,94.497187,134.720089,"Hidden Size=[8], regularizer=0.05, learning_ra..."
1,model_16_8_0,0.999999,0.999063,1.000000,1.000000,1.000000,6.485068e-07,0.000874,6.222161e-14,3.772312e-08,1.886159e-08,0.000045,0.000805,1.000002,0.000840,94.497187,134.720089,"Hidden Size=[8], regularizer=0.05, learning_ra..."
2,model_16_8_11,0.999999,0.999063,1.000000,1.000000,1.000000,6.485068e-07,0.000874,6.222161e-14,3.772312e-08,1.886159e-08,0.000045,0.000805,1.000002,0.000840,94.497187,134.720089,"Hidden Size=[8], regularizer=0.05, learning_ra..."
3,model_16_8_1,0.999999,0.999063,1.000000,1.000000,1.000000,6.485068e-07,0.000874,6.222161e-14,3.772312e-08,1.886159e-08,0.000045,0.000805,1.000002,0.000840,94.497187,134.720089,"Hidden Size=[8], regularizer=0.05, learning_ra..."
4,model_16_8_2,0.999999,0.999063,1.000000,1.000000,1.000000,6.485068e-07,0.000874,6.222161e-14,3.772312e-08,1.886159e-08,0.000045,0.000805,1.000002,0.000840,94.497187,134.720089,"Hidden Size=[8], regularizer=0.05, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1733,model_12_5_14,0.999438,0.998858,1.000000,0.999999,1.000000,5.241626e-04,0.001066,5.095798e-14,4.509330e-07,2.254665e-07,0.000969,0.022895,1.002695,0.023869,73.107417,108.454816,"Hidden Size=[7], regularizer=0.05, learning_ra..."
1734,model_12_5_17,0.999438,0.998858,1.000000,0.999999,1.000000,5.241626e-04,0.001066,5.095798e-14,4.509330e-07,2.254665e-07,0.000969,0.022895,1.002695,0.023869,73.107417,108.454816,"Hidden Size=[7], regularizer=0.05, learning_ra..."
1735,model_12_4_0,0.999438,0.998858,1.000000,0.999999,0.999999,5.241626e-04,0.001066,3.878799e-08,1.090487e-06,5.646376e-07,0.000969,0.022895,1.002695,0.023869,73.107417,108.454816,"Hidden Size=[7], regularizer=0.05, learning_ra..."
1736,model_12_5_15,0.999438,0.998858,1.000000,0.999999,1.000000,5.241626e-04,0.001066,5.095798e-14,4.509330e-07,2.254665e-07,0.000969,0.022895,1.002695,0.023869,73.107417,108.454816,"Hidden Size=[7], regularizer=0.05, learning_ra..."
